In [4]:
import numpy as np
import time

In [2]:
def Lratio(nvec,r,d,p):
    
    n00, n11, n10, n01 = nvec
    s = 0.02
    x1 = ((1-s+d*s*(1-r))/(1-s+d*s))**n00
    x2 = ((r+p-r*p)/p)**n11
    x3 = (1-r)**n10
    x4 = ((p*(1-s)+d*s*(r+p-r*p))/(p*(1-s)+d*s*p))**n01
    x = x1*x2*x3*x4
    return x


def match(r,d,p,itemtype):
    
    s = 0.02
    if itemtype==1:
        rcp = np.random.binomial(1,s,200)
    else:
        rcp = np.zeros(200)
    
    rcp = rcp * np.random.binomial(1,r,200)
    reic = rcp * np.random.binomial(1,(1-d),200)
    prec = np.random.binomial(1,p,200)
    retc = rcp + prec
    retc[retc==1] = 1
    R = reic + retc
    n00 = sum(R==0)
    n11 = sum(R==2)
    pre_n10 = R==1
    n10 = sum(pre_n10[reic==1])
    n01 = sum(R==1)
    n = np.array([n00,n11,n10,n01])
    
    return n


def response(r,d,p,itemtype):
    n = match(r,d,p,itemtype)
    L = Lratio(n,r,d,p)
    return int(L > 1)

def response_vector(r,d,p,Nitems):
    item = np.array([np.tile(1,int(Nitems/2)),np.tile(2,int(Nitems/2))])
    target = [response(r,d,p,i) for i in item[0]]
    distractor = [response(r,d,p,i) for i in item[1]]
    
    return np.c_[target,distractor]


def BCDMEM(pars,Nitems,Ntrials,flag):
    # pars:[r,d,p]
    # r: Learning Rate
    # d: Reinstatement
    # p: Context Noise
    # Ntimes: targets = Nitems/2; distractors = Nitems/2
    # flag: 1, full matrix output; 2, contingency table
    r,d,p = pars
    out1 = np.zeros((Ntrials,int(Nitems/2),2))
    out2 = np.zeros((Ntrials,2,2))
    for n in range(Ntrials):
        o = response_vector(r,d,p,Nitems)
        out1[n,:,:] = o
        out2[n,:,:] = np.array([[sum(o[:,0]),(Nitems/2)-sum(o[:,0])],
                                [sum(o[:,1]),(Nitems/2)-sum(o[:,1])]])
    if flag:
        # first column: vector of Hit (1) and Miss (0) for targets
        # second column: vector of Correct Rejection (0) and False Alarms (1) for distractors
        return out1
    else:
        # first row (target): Hit and Miss
        # second row (distractor): False Alarms and Correct Rejection
        return out2
    

In [11]:
t = time.time()
BCDMEM([0.7,0.1,0.1],40,1,0)
elapsed = time.time() - t

In [12]:
elapsed

0.04491472244262695